<a href="https://colab.research.google.com/github/19K41A05G3/NNDL/blob/main/nndlassignment2_5g3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data = pd.read_excel(os.path.join(os.getcwd(), 'Assets', '/content/drive/MyDrive/assignment_2'))
print(data.shape)
data.head()

(2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [11]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])

In [12]:
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [15]:
maxX = np.max(data['Previous Day Load'])
minX = np.min(data['Previous Day Load'])
maxY = np.max(data['Present Day Load'])
minY = np.min(data['Present Day Load'])

data['Previous Day Load'] = (data['Previous Day Load'] - minX) / (maxX - minX)
data['Present Day Load'] = (data['Present Day Load'] - minY) / (maxY - minY)

In [16]:
x = data['Previous Day Load']
y = data['Present Day Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [17]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
vm = 0
vc = 0
v = 0.5

for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
       
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
    
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        m += vm
        c += vc

print(f'm = {m}\nc = {c}')

m = 0.7171132537325571
c = 0.12511078784145233


In [18]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 0.09759933101621866
MSE : 0.017406654581445538
RMSE : 0.1319342812973396
Testing Error :
MAE : 0.09998259685126545
MSE : 0.016651776248449105
RMSE : 0.12904176164501593


In [19]:
pd.DataFrame({'Actual' : trainY, 'Predicted' : train_pred})

,Actual,Predicted
0,0.154461,0.224829
1,0.774471,0.571979
2,0.451913,0.464937
3,0.236940,0.295842
4,0.824684,0.592111
...,...,...
1939,0.448860,0.421114
1940,0.500128,0.456262
1941,0.349947,0.378354
1942,0.839177,0.761104


In [20]:
pd.DataFrame({'Actual' : testY, 'Predicted' : test_pred})

,Actual,Predicted
0,0.509486,0.419763
1,0.436307,0.644650
2,0.734359,0.552707
3,0.223189,0.451680
4,0.341303,0.472425
...,...,...
211,0.619794,0.752163
212,0.734359,0.612857
213,0.580583,0.579713
214,0.429032,0.400757
